In [ ]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
import configparser

In [ ]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']

### Step 1: Scope the Project and Gather Data

Dados do Governo Brasileiro sobre a [vacinação contra COVID-19](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)

In [ ]:
# Spark session
spark = SparkSession \
        .builder\
        .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .enableHiveSupport().getOrCreate()

In [ ]:
vaccines_df = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

### Step 2: Explore and Assess the Data

In [ ]:
vaccines_df.dtypes

In [ ]:
vaccines_df.select(
    "estabelecimento_municipio_nome", 
    "vacina_fabricante_nome", "vacina_nome",
    "estalecimento_nofantasia").filter(
        vaccines_df.estabelecimento_municipio_nome == 'PINHEIRO'
        ).show(10)